In [2]:
import graphlab as gl
import graphlab.aggregate as agg
from graphlab.toolkits.feature_engineering import NumericImputer
import numpy as np
from sklearn.decomposition import PCA
import pickle
import pandas as pd
import os
import datetime as dt

This non-commercial license of GraphLab Create for academic use is assigned to fella.belkham@u-psud.fr and will expire on December 27, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1483555248.log


In [ ]:
gpCalls = pickle.load(open("gpCalls.obj", "rb"))

In [ ]:
testCalls = pickle.load(open("testCalls.obj", "rb"))

In [42]:
pickle.dump(gpCalls, open("gpCalls.obj", "wb"))

In [43]:
pickle.dump(testCalls, open("testCalls.obj", "wb"))

In [6]:
print ('Loading the training set ..')

columns =  ['DATE','WEEK_END', 'DAY_WE_DS', 'TPER_TEAM',
            'ASS_ASSIGNMENT', 'ASS_PARTNER', 'ASS_POLE',
            'CSPL_ABNCALLS','CSPL_MAXINQUEUE', 'CSPL_ACCEPTABLE', 'CSPL_I_ACDTIME',
            'CSPL_ABANDONNED_CALLS', 'CSPL_CALLS', 'CSPL_RECEIVED_CALLS'
           ]
url = 'train_2011_2012_2013.csv'
callsData = gl.SFrame.read_csv(url, delimiter=';', header=True, usecols= columns)

Loading the training set ..


Finished parsing file /home/fella/train_2011_2012_2013.csv

Parsing completed. Parsed 100 lines in 2.89153 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,str,str,str,str,str,int,int,int,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 163398 lines. Lines per second: 55710.3

...

Read 9290581 lines. Lines per second: 122589

Read 9943248 lines. Lines per second: 122607

Read 10595759 lines. Lines per second: 122643

Finished parsing file /home/fella/train_2011_2012_2013.csv

Parsing completed. Parsed 10878470 lines in 87.9582 secs.

In [7]:
print "Loading test set"
test=gl.SFrame.read_csv('submission.txt', delimiter='\t', verbose=False)
test=test['DATE', 'ASS_ASSIGNMENT']

Loading test set


In [8]:
print('Splitting column DATE into columns year, month, day, hour, minute.. in the Training set')      

date_col=callsData['DATE'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.000'))
splitted_date_col=date_col.split_datetime(column_name_prefix='', limit=['year', 'month', 'day', 'hour', 'minute'])
for col in ['year', 'month', 'day', 'hour', 'minute']:
    callsData[col] = splitted_date_col[col]

print('Splitting column DATE into columns year, month, day, hour, minute.. In the Test set')      
date_col2=test['DATE'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.000'))
splitted_date_col=date_col2.split_datetime(column_name_prefix='', limit=['year', 'month', 'day', 'hour', 'minute'])
for col in ['year', 'month', 'day', 'hour', 'minute']:
    test[col] = splitted_date_col[col]

Splitting column DATE into columns year, month, day, hour, minute.. in the Training set
Splitting column DATE into columns year, month, day, hour, minute.. In the Test set


In [9]:
calls = callsData.to_dataframe()

In [11]:
testCalls = test.to_dataframe()

In [12]:
gpCalls = calls.groupby(['DATE','WEEK_END', 'DAY_WE_DS', 'ASS_ASSIGNMENT', 'TPER_TEAM','year', 'month', 'day', 'hour', 'minute']).sum().reset_index()

In [13]:
gpCalls

,DATE,WEEK_END,DAY_WE_DS,ASS_ASSIGNMENT,TPER_TEAM,year,month,day,hour,minute,CSPL_ABNCALLS,CSPL_MAXINQUEUE,CSPL_ACCEPTABLE,CSPL_I_ACDTIME,CSPL_ABANDONNED_CALLS,CSPL_CALLS,CSPL_RECEIVED_CALLS
0,2011-01-01 00:00:00.000,1,Samedi,Crises,Nuit,2011,1,1,0,0,0,0,0,0,0,0,0
1,2011-01-01 00:00:00.000,1,Samedi,Domicile,Nuit,2011,1,1,0,0,0,0,0,0,0,0,0
2,2011-01-01 00:00:00.000,1,Samedi,Gestion,Nuit,2011,1,1,0,0,0,0,0,0,0,0,0
3,2011-01-01 00:00:00.000,1,Samedi,Gestion - Accueil Telephonique,Nuit,2011,1,1,0,0,0,0,0,0,0,0,0
4,2011-01-01 00:00:00.000,1,Samedi,Gestion Amex,Nuit,2011,1,1,0,0,0,0,0,0,0,0,0
5,2011-01-01 00:00:00.000,1,Samedi,Gestion Assurances,Nuit,2011,1,1,0,0,0,0,0,0,0,0,0
6,2011-01-01 00:00:00.000,1,Samedi,Gestion Clients,Nuit,2011,1,1,0,0,0,0,0,0,0,0,0
7,2011-01-01 00:00:00.000,1,Samedi,Gestion Renault,Nuit,2011,1,1,0,0,0,0,0,0,0,0,0
8,2011-01-01 00:00:00.000,1,Samedi,Japon,Nuit,2011,1,1,0,0,0,0,0,0,0,0,0
9,2011-01-01 00:00:00.000,1,Samedi,Manager,Nuit,2011,1,1,0,0,0,0,0,0,0,0,0


In [14]:
np.sort(calls['CSPL_RECEIVED_CALLS'].unique())

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 179, 180, 18

In [15]:
np.sort(gpCalls['CSPL_RECEIVED_CALLS'].unique())

array([   0,    1,    2, ..., 1343, 1345, 1365])

In [16]:
verif = gpCalls.groupby(['DATE', 'ASS_ASSIGNMENT']).count().reset_index()

In [17]:
verif[verif['CSPL_RECEIVED_CALLS']>1]
# should be DataFrame empty 

,DATE,ASS_ASSIGNMENT,WEEK_END,DAY_WE_DS,TPER_TEAM,year,month,day,hour,minute,CSPL_ABNCALLS,CSPL_MAXINQUEUE,CSPL_ACCEPTABLE,CSPL_I_ACDTIME,CSPL_ABANDONNED_CALLS,CSPL_CALLS,CSPL_RECEIVED_CALLS


In [18]:
print "Transform the categories : ASS_ASSIGNMENT to ID..."

ass=gpCalls['ASS_ASSIGNMENT'].unique()
ass_numpy=ass #.to_numpy() -- add it if sframe
gpCalls['cod_ASS_ASSIGNMENT']=gpCalls['ASS_ASSIGNMENT'].apply(lambda x: np.where(ass_numpy==x)[0][0])
testCalls['cod_ASS_ASSIGNMENT']=testCalls['ASS_ASSIGNMENT'].apply(lambda x: np.where(ass_numpy==x)[0][0])


Transform the categories : ASS_ASSIGNMENT to ID...


In [75]:
print "Transform the categories : TPER_TEAM to ID..."

equipe=gpCalls['TPER_TEAM'].unique()
equipe_numpy=equipe #.to_numpy() -- add it if sframe
gpCalls['cod_TPER_TEAM']=gpCalls['TPER_TEAM'].apply(lambda x: np.where(equipe==x)[0][0])


In [19]:
print "Adding colum, Day to Test set"
testCalls['DAY_WE_DS'] = date_col2.apply(lambda x: x.weekday())

Adding colum, Day to Test set


In [48]:
print 'Adding weekend column to Test set'
testCalls['WEEK_END'] =np.where(testCalls['DAY_WE_DS']>= 5, 1,0)

Adding weekend column to Test set


In [21]:
print('Changing day label column into numerical in the right order (Monday=0,Thuesday=1,...)..')   
#update date col 
up_date_col=gpCalls['DATE'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.000'))
gpCalls['DAY_WE_DS'] = up_date_col.apply(lambda x: x.weekday())

Changing day label column into numerical in the right order (Monday=0,Thuesday=1,...)..


### Ploting graph to find features



### Adding Features

In [6]:
# 5 NIVEAU DE PIQUES
# Niveau de pique par heure if not weekend
wd_niv_0 = [0, 1, 2, 3, 4, 5, 6, 7, 21, 22, 23]
wd_niv_1 = [8, 19, 20]
wd_niv_2 = [12, 13,18 ]
wd_niv_3 = [9, 14 ,15, 16 ,17]
wd_niv_4 = [10, 11]

# Niveau de pique par heure if weekend 
we_niv_0 = [0, 1, 2, 3, 4, 5, 6, 7, 8,  21, 22 ,23]
we_niv_1 = [9 ,10 ,12 ,13 ,14 ,15, 16 ,17, 18, 19, 20 ]
we_niv_2 = [11]


In [33]:
gpCalls['PIQUE_NV'] = np.where((gpCalls['WEEK_END']==0) & (gpCalls['hour'].isin(wd_niv_0) ), 0, None)
gpCalls['PIQUE_NV'] = np.where((gpCalls['WEEK_END']==0) & (gpCalls['hour'].isin(wd_niv_1) ), 1, gpCalls['PIQUE_NV'])
gpCalls['PIQUE_NV'] = np.where((gpCalls['WEEK_END']==0) & (gpCalls['hour'].isin(wd_niv_2) ), 2, gpCalls['PIQUE_NV'])
gpCalls['PIQUE_NV'] = np.where((gpCalls['WEEK_END']==0) & (gpCalls['hour'].isin(wd_niv_3) ), 3, gpCalls['PIQUE_NV'])
gpCalls['PIQUE_NV'] = np.where((gpCalls['WEEK_END']==0) & (gpCalls['hour'].isin(wd_niv_4) ), 4, gpCalls['PIQUE_NV'])
gpCalls['PIQUE_NV'] = np.where((gpCalls['WEEK_END']==1) & (gpCalls['hour'].isin(we_niv_0) ), 0, gpCalls['PIQUE_NV'])
gpCalls['PIQUE_NV'] = np.where((gpCalls['WEEK_END']==1) & (gpCalls['hour'].isin(we_niv_1) ), 1, gpCalls['PIQUE_NV'])
gpCalls['PIQUE_NV'] = np.where((gpCalls['WEEK_END']==1) & (gpCalls['hour'].isin(we_niv_2) ), 2,gpCalls['PIQUE_NV'])
# ---------------------------------#
testCalls['PIQUE_NV'] = np.where((testCalls['WEEK_END']==0) & (testCalls['hour'].isin(wd_niv_0) ), 0, None)
testCalls['PIQUE_NV'] = np.where((testCalls['WEEK_END']==0) & (testCalls['hour'].isin(wd_niv_1) ), 1, testCalls['PIQUE_NV'])
testCalls['PIQUE_NV'] = np.where((testCalls['WEEK_END']==0) & (testCalls['hour'].isin(wd_niv_2) ), 2, testCalls['PIQUE_NV'])
testCalls['PIQUE_NV'] = np.where((testCalls['WEEK_END']==0) & (testCalls['hour'].isin(wd_niv_3) ), 3, testCalls['PIQUE_NV'])
testCalls['PIQUE_NV'] = np.where((testCalls['WEEK_END']==0) & (testCalls['hour'].isin(wd_niv_4) ), 4, testCalls['PIQUE_NV'])
testCalls['PIQUE_NV'] = np.where((testCalls['WEEK_END']==1) & (testCalls['hour'].isin(we_niv_0) ), 0, testCalls['PIQUE_NV'])
testCalls['PIQUE_NV'] = np.where((testCalls['WEEK_END']==1) & (testCalls['hour'].isin(we_niv_1) ), 1, testCalls['PIQUE_NV'])
testCalls['PIQUE_NV'] = np.where((testCalls['WEEK_END']==1) & (testCalls['hour'].isin(we_niv_2) ), 2, testCalls['PIQUE_NV'])

In [38]:
# NIVEAU DE PIQUE BINAIRE
# Niveau de pique par heure if not weekend
pique_niv1 = [9,10,11,14,15,16,17]
gpCalls['PIQUE_Bin'] = np.where((gpCalls['WEEK_END']==0) & (gpCalls['hour'].isin(pique_niv1) ), 1, 0)
testCalls['PIQUE_Bin'] = np.where((testCalls['WEEK_END']==0) & (testCalls['hour'].isin(pique_niv1) ), 1, 0)


In [82]:
print 'Adding COD_TPER_TEAM column to Test set'
testCalls['cod_TPER_TEAM'] = np.where(((testCalls['hour']<=6) | ((testCalls['hour']==23) & (testCalls['minute']>=30)) | ((testCalls['hour']==7) & (testCalls['minute']<=30))), 0, 1)